In [ ]:
import veloxchem as vlx
import adcc
from mpi4py import MPI
import sys
import numpy
from scipy.sparse import linalg
from veloxchem.qqscheme import get_qq_scheme
from veloxchem.veloxchemlib import fockmat

numpy.set_printoptions(precision=7, suppress=True)

### Prepare a VeloxChem SCF calculation

In [ ]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 3, 'n_state_deriv': 1} #

In [ ]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [ ]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

In [ ]:
# SCF
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)

In [ ]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

### Excited-State Calculation via CIS (TDHF/TDA)

In [ ]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

In [ ]:
# Get MO coefficients

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

In [ ]:
# Get the overlap and Fock matrices
#print(scfdrv.scf_tensors.keys())
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
# Transform them from AO to MO basis
# print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
# print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

In [ ]:
# Get the CIS eigenvalues and eigenvectors
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

## Use the new OrbitalResponse module

\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a ) = 
   R_{ia}
   %R^{\text{CIS}}_{ia}
\end{equation}

In [ ]:
from veloxchem.orbitalresponse import OrbitalResponse

orb_resp = OrbitalResponse(comm, ostream)
orb_resp.update_settings(rsp_settings, method_settings)
orb_rsp_results = orb_resp.compute(molecule, basis, scfdrv.scf_tensors, tda_eig_vecs)

print(orb_rsp_results.keys())

print()
print("lambda multipliers in MO basis:")
print(orb_rsp_results['lambda_multipliers'])
print()
print("lambda multipliers in AO basis:")
print(orb_rsp_results['lambda_multipliers_ao'])
print()
print("omega multipliers in AO basis:")
print(orb_rsp_results['omega_multipliers'])

### Calculate the "gradient" and first-order properties

\begin{equation}
    \vec{\mu} = \frac{\partial L}{\partial \vec{F}} = \sum_{ij} \gamma_{ij} \, \mu_{ji} + \sum_{ab} \gamma_{ab} \, \mu_{ba} + 2 \sum_{ia} \lambda_{ia} \, \mu_{ai}
\end{equation}
where
$\mu_{pq} = \langle \phi_p | q \vec{r} | \phi_q \rangle %= \int \phi_p q \vec{r} \phi_q d\vec{r}$

In [ ]:
from veloxchem.tdagradientdriver import TdaGradientDriver

tdagrad_drv = TdaGradientDriver(comm, ostream, scfdrv, tda_results)
tdagrad_drv.update_settings(rsp_settings, method_settings)
#print(tdagrad_drv.n_state_deriv)
#tdagrad_drv.compute(molecule, basis)
tda_prop = tdagrad_drv.compute_properties(molecule, basis, scfdrv.scf_tensors, orb_rsp_results)

tdagrad_drv.print_properties(molecule, tda_prop)

print()
print("First excited state dipole moments:")
print("ADC(1)/STO-3G unrelaxed dipole moment [a.u.] for water in adcc:")
print("[-0.2107819, -0.,     -0.1423891]")
print()
print("CIS/STO-3G relaxed dipole moment [a.u.] for water in Q-Chem:")
print("[-0.0184062, 0.00000, -0.0076203]")
print("Total dipole: 0.020 a.u., 0.051 Debye")

### Calculate one- and two-particle density matrices

\begin{align}
    \gamma_{ij} &= - \sum_{a} x_{ia} x_{ja} \\
    \gamma_{ab} &= + \sum_{i} x_{ia} x_{ib} \\
    \Gamma_{iajb} &= - x_{ib} x_{ja}
\end{align}

In [ ]:
# VeloxChem unrelaxed one-particle density matrix and two-particle density matrix

vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

# print(vlx_dm_oo)

### Transform the one-particle density matrices from the MO to the AO basis

\begin{equation}
    \gamma_{\mu \nu} = \sum_{pq} C_{\mu p} \gamma_{pq} C_{\nu q}
\end{equation}

In [ ]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao

# print("VLX:")
# print(vlx_dm_ao)

#### Transform the excitation vectors to the AO basis
\begin{equation}
    x_{\mu \nu} = \sum_{ia} C_{\mu i} x_{ia} C_{\nu a}
\end{equation}

In [ ]:
# Transform the transition density matrix (the excitation vectors) from MO to AO basis
# (will be needed for the two-particle density matrix)
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
# print(tda_eig_vec_ao)

The MO to AO transformation of the contribution of the two-particle density matrix with the ERI tensor is given as
\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

# Orbital response

\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a ) = 
   R_{ia}
   %R^{\text{CIS}}_{ia}
\end{equation}

## Right-hand side

### 1. Contributions from the 1PDMs
\begin{align}
F^\gamma_{ia} &= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a} 2F^{1,\alpha}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{align}

### 2. Contributions from the 2PDMs

\begin{align}
F^\Gamma_{ia} &= ~~ \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu) \Big] \\
&~~~ -\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[ \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\theta\nu}(\mu\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi) \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\nu\theta}(\zeta\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ \sum_{\theta,\nu} x^\alpha_{\theta\nu} \big[ 2 (\mu\varphi|\theta\nu)- (\mu\nu|\theta\varphi) \big] \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ \sum_{\theta,\nu} x^\alpha_{\nu\theta} \big[ 2(\zeta\varphi|\theta\nu)- (\zeta\nu|\theta\varphi) \big] \Big]
\end{align}

In [ ]:
# Calculate two different parts and transform the sum to MO
nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv_pqrs = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv_pqrs.compute_in_mem(molecule, basis, pqrs)

F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
# The density matrices already have a factor of 2 (for alpha and beta spin)
# hence a factor of 0.5 where the other spin part actually vanishes
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

# Start -- Orbital Response 

In [ ]:
# Set up ERI driver:
nao = mo.shape[0]
#Probably these will be properties of our OrbitalResponseSolver?
qq_type = 'QQ_DEN'
eri_thresh = 1.0e-15
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
screening = eri_drv.compute(get_qq_scheme(qq_type),
                            eri_thresh, molecule, basis)
# print(dir(screening))
# print(screening.get_screener(7))


In [ ]:
# Use AODensityMatrix object to compute a Fock-like matrix
# expect something like F_{\varphi\theta}, equation above
# see linearsolver.py; we have to create a linear solver object:
# 
# from veloxchem.linearsolver import LinearSolver
from veloxchem.veloxchemlib import AOFockMatrix
from veloxchem.veloxchemlib import AODensityMatrix
from veloxchem.veloxchemlib import denmat

#Create all AODensityMatrices needed for the RHS
#1PDM contribution:
ao_density_1pdm = AODensityMatrix([vlx_dm_ao], denmat.rest)

#2PDM contribution
ao_density_2pdm_1 = AODensityMatrix([tda_eig_vec_ao.T], denmat.rest)
ao_density_2pdm_2 = AODensityMatrix([tda_eig_vec_ao], denmat.rest)

#We only use one of the terms (i.e. ao_density_2pdm_2 -- coresponding to tda_eig_vec_ao)
#because we can get the other as its transpose
ao_density_RHS_all = AODensityMatrix([vlx_dm_ao, tda_eig_vec_ao], denmat.rest)

print("Single terms:")
print("1PDM")
print(ao_density_1pdm)
print()
print("2PDM:")
print(ao_density_2pdm_1)
print()
print(ao_density_2pdm_2)

print("ALL RHS:")
print(ao_density_RHS_all)

In [ ]:
fock_rhs_1pdm = AOFockMatrix(ao_density_1pdm)
fock_rhs_2pdm_1 = AOFockMatrix(ao_density_2pdm_1)
fock_rhs_2pdm_2 = AOFockMatrix(ao_density_2pdm_2)

fock_RHS_all = AOFockMatrix(ao_density_RHS_all)
fock_flag = fockmat.rgenjk
for i in range(fock_RHS_all.number_of_fock_matrices()):
    fock_RHS_all.set_fock_type(fock_flag, i)
    
fock_rhs_2pdm_1.set_fock_type(fock_flag, 0)
fock_rhs_2pdm_2.set_fock_type(fock_flag, 0)

eri_drv.compute(fock_rhs_1pdm, ao_density_1pdm, molecule, basis, screening) 
eri_drv.compute(fock_rhs_2pdm_1, ao_density_2pdm_1, molecule, basis, screening) 
eri_drv.compute(fock_rhs_2pdm_2, ao_density_2pdm_2, molecule, basis, screening) 

eri_drv.compute(fock_RHS_all, ao_density_RHS_all, molecule, basis, screening)

print("Fock matrices, single terms:")
print("Fock-1PDM")
print(fock_rhs_1pdm)
print()
print("Fock-2PDM:")
print(fock_rhs_2pdm_1)
print()
print(fock_rhs_2pdm_2)

print("Fock-ALL RHS:")
print(fock_RHS_all)



In [ ]:
#Transform the 1PDM contribution to the RHS to MO basis:
MO_rhs_1pdm = numpy.einsum('pi,pa->ia', mo_occ,
                    numpy.einsum('ta,pt->pa', mo_vir, 0.5*fock_RHS_all.alpha_to_numpy(0)))

print("\nIn MO basis, 1PDM contribution to the RHS, using one-shot Fock-build?:")
print(MO_rhs_1pdm)
print()
print("Original, for comparison:")
print(vlx_1pdm_rhs)

In [ ]:
#Contract with excitation vector, overlap integrals and transform the 2PDM contribution to the RHS to MO basis:
#print(fock_RHS_all.alpha_to_numpy(1))

#LR_2pdm_F1 =  numpy.einsum('rz,mr->mz', ovlp,
#                numpy.einsum('pr,mp->mr', tda_eig_vec_ao, 0.5*fock_RHS_all.alpha_to_numpy(1).T))
#LR_2pdm_F2 =  numpy.einsum('mr,rz->mz', ovlp,
#                numpy.einsum('rp,zp->rz', tda_eig_vec_ao, 0.5*fock_RHS_all.alpha_to_numpy(1)))

MO_rhs_2pdm = numpy.einsum('mi,ma->ia', mo_occ, 
                                 numpy.einsum('za,mz->ma', mo_vir,  
                                              (numpy.einsum('mr,rz->mz', ovlp,
                                                            numpy.einsum('rp,zp->rz',
                                                                         tda_eig_vec_ao, 
                                                                         0.5*fock_RHS_all.alpha_to_numpy(1))))
                                              
                                              -(numpy.einsum('rz,mr->mz', ovlp,
                                                             numpy.einsum('pr,mp->mr',
                                                                          tda_eig_vec_ao,
                                                                          0.5*fock_RHS_all.alpha_to_numpy(1).T))))
                          )

print(MO_rhs_2pdm)

In [ ]:
RHS = MO_rhs_1pdm+MO_rhs_2pdm

## Left-hand side
### Initial guess for Lagrange multipliers $\boldsymbol{\lambda}$

\begin{equation}
    \lambda_{ia}^{(0)} = \frac{R_{ia}}{\varepsilon_i - \varepsilon_a}
\end{equation}

In [ ]:
#Construct Initial Guess: lambda_guess=RHS/(epsilon_i-epsilon_a)
orb_ene = scfdrv.scf_tensors['E']
eocc = orb_ene[:nocc]
evir = orb_ene[nocc:]
eov = -evir + eocc.reshape(-1, 1)
#print(eov)

lambda_guess = RHS / eov

print(lambda_guess)

### Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$ and assuming real orbitals, the equation is obtained as
\begin{align}
-\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    &= -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big] \\
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
    &= - \sum_{\varphi \theta} \big[ C_{\varphi i} C_{\theta a} + C_{\theta i} C_{\varphi a} \big]
        \sum_{\mu \nu} \lambda_{\mu \nu} \big[ 2 (\mu \nu | \varphi \theta) - (\mu \theta | \varphi \nu) \big]
\end{align}

#### Full LHS 
\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a )
\end{equation}

## Solution via Conjugate Gradient

To solve: $\mathbf{A} \mathbf{x} = \mathbf{b}$

However, we do not "know" the matrix $\mathbf{A}$, only the matrix-vector product $\mathbf{Ax}$.
Thus, define it as a linear operator with a vector $\mathbf{x}$ as an argument.

### Defining a linear operator

In [ ]:
lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
    
#Create AODensityMatrix object from lambda in AO
ao_density_lambda_guess = AODensityMatrix([lambda_guess_ao], denmat.rest)
fock_lambda = AOFockMatrix(ao_density_lambda_guess)
fock_lambda.set_fock_type(fock_flag, 0)

#Matrix-vector product routine for conjugate gradient:
def OrbRsp_LHS_mvp(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix, using AODensityMatrix, AOFockMatrix
    
    v: lambda at current iteration"""
    
    #lambda_mo = v.reshape(nocc,nvir)
    
    # Transform to AO
    lambda_ao = numpy.matmul(mo_occ, numpy.matmul(v.reshape(nocc,nvir), mo_vir.T))
    
    #Create AODensityMatrix object from lambda in AO
    ao_density_lambda = AODensityMatrix([lambda_ao], denmat.rest)
    
    #Create a Fock Matrix Object (initialized with zeros)    
    eri_drv.compute(fock_lambda, ao_density_lambda, molecule, basis, screening) 
    
    #Transform to MO basis (symmetrized w.r.t. occ. and virt.) and add diagonal part
    lambda_mo = (-( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_vir)) 
                + numpy.matmul(mo_vir.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_occ)).T )
                +v.reshape(nocc,nvir) * eov) 
    
    return lambda_mo.reshape(nocc*nvir)

In [ ]:
#OrbRsp_LHS_mvp(lambda_guess.reshape(nocc*nvir))
A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=OrbRsp_LHS_mvp)
solution, w = linalg.cg(A=A, b=RHS.reshape(nocc*nvir), x0=lambda_guess.reshape(nocc*nvir), tol=1e-8, maxiter=25)

lambda_OV = solution.reshape(nocc,nvir)
print(lambda_OV)

# Omega Multipliers (for Overlap Matrix)

## General Equation:
\begin{equation}
2\left(\gamma'_{ut}+\lambda_{ut}\right)\epsilon_u +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)\langle{pu||qt}\rangle\delta_{t\epsilon_o}+\sum_{p,q,r}\Gamma'_{tpqr}\langle{up||qr}\rangle+2\omega_{ut}=0\
\end{equation}

## VV block ($\alpha$ block)
\begin{equation}
2\gamma'_{ab}\epsilon_a +\sum_{p,q,r}\Gamma'_{bpqr}\langle{ap||qr}\rangle+2\omega_{ab}=0\\
2\gamma'_{ab}\epsilon_a +\sum_{p,q,r}\Gamma'_{bpqr}([aq|pr]-[ar|pq])+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a +\sum_{p,q,r}\Gamma'_{bpqr}[(aq|pr)(\alpha\sigma'|\sigma\sigma'')-(ar|pq)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a +\sum_{p,q,r}\Gamma'_{pbrq}[(aq|pr)(\alpha\sigma'|\sigma\sigma'')-(ar|pq)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a +2\sum_{i,c,j}\Gamma'_{ibjc}[(ac|ij)(\alpha\sigma'|\sigma\sigma'')-(aj|ic)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a -2\sum_{i,c,j}x_{ic}x_{jb}[(ac|ij)(\alpha\sigma'|\sigma\sigma'')-(aj|ic)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a -2\sum_{i,c,j}x_{ic}x_{jb}[(ac|ij)(\alpha\alpha|\alpha\alpha)-(aj|ic)(\alpha\alpha|\sigma\sigma)]+2\omega_{ab} = 0\\
2\gamma'_{ab}\epsilon_a + 2\sum_{i,c,j}x_{ic}x_{jb}[2(aj|ic)-(ac|ij)]+2\omega_{ab} = 0\\
\gamma'_{ab}\epsilon_a + \sum_{\mu\nu\theta\varphi}C_{\theta a}[2(\theta\nu|\mu\varphi)-(\theta\varphi|\mu\nu)]\sum_{j}x_{jb}C_{\nu j}\sum_{i,c}C_{\mu i}x_{ic}C_{\varphi c}+\omega_{ab} = 0\\
\gamma'_{ab}\epsilon_a + \sum_{\mu\nu\theta\varphi}C_{\theta a}[2(\theta\nu|\mu\varphi)-(\theta\varphi|\mu\nu)]\sum_{j}C_{\nu j}x_{jb}x_{\mu\varphi}+\omega_{ab} = 0\\
\gamma'_{ab}\epsilon_a + \sum_{\mu\nu\theta\varphi}C_{\theta a}[2(\theta\nu|\mu\varphi)-(\theta\varphi|\mu\nu)]x_{\nu b}x_{\mu\varphi}+\omega_{ab} = 0\\
\gamma'_{ab}\epsilon_a + \sum_{\mu\nu\theta\varphi}C_{\theta a}[2(\theta\nu|\mu\varphi)-(\theta\varphi|\mu\nu)]\sum_{\rho,\zeta}x_{\nu \zeta}S_{\zeta\rho}C_{\rho b}x_{\mu\varphi}+\omega_{ab} = 0\\
\gamma'_{ab}\epsilon_a + \sum_{\theta\rho}C_{\theta a}C_{\rho b}\sum_{\nu,\zeta}x_{\nu \zeta}S_{\zeta\rho} \sum_{\mu,\varphi}[2(\theta\nu|\mu\varphi)-(\theta\varphi|\mu\nu)]x_{\mu\varphi}+\omega_{ab} = 0\\
%
\gamma'_{ab}\epsilon_a = \gamma'_{ab} (F_{ab} \delta_{ab})
\end{equation}


In [ ]:
#omega_VV = numpy.zeros(vlx_dm_vv.shape)
omega_VV = -numpy.einsum('ta,tb->ab', mo_vir, 
                                 numpy.einsum('rb,tr->tb', mo_vir,  
                                              (numpy.einsum('zr,tz->tr', ovlp,
                                                            numpy.einsum('nz,tn->tz',
                                                                         tda_eig_vec_ao, 
                                                                         0.5*fock_RHS_all.alpha_to_numpy(1).T
                                                                        )
                                                           )
                                              )
                                             )
                        )
# print("MO virtual energies:")
# print(evir)
# print()
omega_VV_wo_e = omega_VV
for a in range(evir.shape[0]):
    omega_VV[a,:]-=evir[a]*0.5*vlx_dm_vv[a,:]
print(omega_VV)
# print("1PDM")
# print(vlx_dm_vv/2)

## OV block ($\alpha$ block)
\begin{equation}
2\lambda_{ia}\epsilon_i + \sum_{p,q,r}\Gamma'_{apqr}\langle{ip||qr}\rangle+2\omega_{ia}=0\\
2\lambda_{ia}\epsilon_i + \sum_{p,q,r}\Gamma'_{parq}[(iq|pr)(\alpha\sigma'|\sigma\sigma'')-(ir|pq)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ia}=0\\
2\lambda_{ia}\epsilon_i + 2\sum_{j,b,k}\Gamma'_{jakb}[(ib|jk)(\alpha\sigma'|\sigma\sigma'')-(ik|jb)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ia}=0\\
\lambda_{ia}\epsilon_i + \sum_{j,b,k}x_{jb}x_{ka}[2(ik|jb)-(ib|jk)]+\omega_{ia}=0\\
\lambda_{ia}\epsilon_i + \sum_{\mu,\nu,\theta,\varphi}\sum_{j,b,k}C_{\mu i}C_{\nu j}C_{\theta k}C_{\varphi b}x_{jb}x_{ka}[2(\mu\theta|\nu\varphi)-(\mu\varphi|\nu\theta)]+\omega_{ia}=0\\
\lambda_{ia}\epsilon_i + \sum_{\mu,\nu,\theta,\varphi}C_{\mu i}x_{\nu\varphi}\sum_{\rho,\zeta}x_{\theta \rho}S_{\rho\zeta}C_{\zeta a}[2(\mu\theta|\nu\varphi)-(\mu\varphi|\nu\theta)]+\omega_{ia}=0\\
\lambda_{ia}\epsilon_i + \sum_{\mu, \zeta}C_{\mu i}C_{\zeta a}\sum_{\rho,\theta}x_{\theta \rho}S_{\rho\zeta}\sum_{\nu,\varphi}[2(\mu\theta|\nu\varphi)-(\mu\varphi|\nu\theta)]x_{\nu\varphi}+\omega_{ia}=0\\
\end{equation}

In [ ]:
omega_OV = numpy.zeros((nocc,nvir))
omega_OV = -numpy.einsum('mi,ma->ia', mo_occ, 
                                 numpy.einsum('za,mz->ma', mo_vir,  
                                              (numpy.einsum('rz,mr->mz', ovlp,
                                                            numpy.einsum('tr,mt->mr',
                                                                         tda_eig_vec_ao, 
                                                                         0.5*fock_RHS_all.alpha_to_numpy(1).T
                                                                        )
                                                           )
                                              )
                                             )
                        )
# print("MO occupied energies:")
#print(eocc)
# print()
for i in range(nocc):
    omega_OV[i,:]-=eocc[i]*lambda_OV[i,:]
print(omega_OV)

## OO block ($\alpha$ block)
\begin{equation}
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)\langle{pi||qj}\rangle+\sum_{p,q,r}\Gamma'_{jpqr}\langle{ip||qr}\rangle+2\omega_{ij}=0\\
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)([pq|ij]-[pj|iq])+\sum_{p,q,r}\Gamma'_{jpqr}([iq|pr]-[ir|pq])+2\omega_{ij}=0\\
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)[(pq|ij)(\sigma\sigma'|\alpha\alpha)-(pj|iq)(\sigma\alpha|\alpha\sigma')]+\sum_{p,q,r}\Gamma'_{jpqr}[(iq|pr)(\alpha\sigma'|\sigma\sigma'')-(ir|pq)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ij}=0\\
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)[2(pq|ij)-(pj|iq)]+\sum_{p,q,r}\Gamma'_{jpqr}[(iq|pr)(\alpha\sigma'|\sigma\sigma'')-(ir|pq)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ij}=0\\
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)[2(pq|ij)-(pj|iq)]+2\sum_{a,k,b}\Gamma'_{jakb}[(ik|ab)(\alpha\sigma'|\sigma\sigma'')-(ib|ak)(\alpha\sigma''|\sigma\sigma')]+2\omega_{ij}=0\\
2\gamma'_{ij}\epsilon_i +2\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)[2(pq|ij)-(pj|iq)]-2\sum_{a,k,b}x_{jb}x_{ka}[(ik|ab)-2(ib|ak)]+2\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{p,q}\left(\gamma'_{pq}+\lambda_{pq}\right)[2(pq|ij)-(pj|iq)]+\sum_{a,k,b}x_{jb}x_{ka}[2(ib|ak)-(ik|ab)]+\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{\mu,\nu,\theta,\varphi}\sum_{p,q}C_{\mu p}C_{\nu q}C_{\theta i}C_{\varphi j}\left(\gamma'_{pq}+\lambda_{pq}\right)[2(\mu\nu|\theta\varphi)-(\mu\varphi|\theta\nu)]+\sum_{a,k,b}x_{jb}x_{ka}[2(ib|ak)-(ik|ab)]+\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{\theta,\varphi}C_{\theta i}C_{\varphi j}\sum_{\mu,\nu}\left(\gamma'_{\mu\nu}+\lambda_{\mu\nu}\right)[2(\mu\nu|\theta\varphi)-(\mu\varphi|\theta\nu)]+\sum_{\mu,\nu,\theta,\varphi}\sum_{a,k,b}C_{\mu i}C_{\nu k}C_{\theta a}C_{\varphi b}x_{jb}x_{ka}[2(\mu\varphi|\theta\nu)-(\mu\nu|\theta\varphi)]+\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{\theta,\varphi}C_{\theta i}C_{\varphi j}\sum_{\mu,\nu}\left(\gamma'_{\mu\nu}+\lambda_{\mu\nu}\right)[2(\mu\nu|\theta\varphi)-(\mu\varphi|\theta\nu)]+\sum_{\mu}C_{\mu i}\sum_{\nu,\theta,\varphi}x_{j\varphi}x_{\nu\theta}[2(\mu\varphi|\theta\nu)-(\mu\nu|\theta\varphi)]+\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{\theta,\varphi}C_{\theta i}C_{\varphi j}\sum_{\mu,\nu}\left(\gamma'_{\mu\nu}+\lambda_{\mu\nu}\right)[2(\mu\nu|\theta\varphi)-(\mu\varphi|\theta\nu)]+\sum_{\mu}C_{\mu i}\sum_{\nu,\theta,\varphi,\rho,\zeta}C_{\zeta j}S_{\zeta\rho}x_{\rho\varphi}x_{\nu\theta}[2(\mu\varphi|\theta\nu)-(\mu\nu|\theta\varphi)]+\omega_{ij}=0\\
\gamma'_{ij}\epsilon_i +\sum_{\theta,\varphi}C_{\theta i}C_{\varphi j}\sum_{\mu,\nu}\left(\gamma'_{\mu\nu}+\lambda_{\mu\nu}\right)[2(\mu\nu|\theta\varphi)-(\mu\varphi|\theta\nu)]+\sum_{\mu,\zeta}C_{\mu i}C_{\zeta j}\sum_{\varphi,\rho}S_{\zeta\rho}x_{\rho\varphi}\sum_{\nu,\theta}[2(\mu\varphi|\theta\nu)-(\mu\nu|\theta\varphi)]x_{\nu\theta}+\omega_{ij}=0\\
\end{equation}

In [ ]:
omega_OO = numpy.zeros((nocc,nocc))
omega_OO = -numpy.einsum('mi,mj->ij', mo_occ, 
                                 numpy.einsum('zj,mz->mj', mo_occ,  
                                              (numpy.einsum('zr,mr->mz', ovlp,
                                                            numpy.einsum('rp,mp->mr',
                                                                         tda_eig_vec_ao, 
                                                                         0.5*fock_RHS_all.alpha_to_numpy(1)
                                                                        )
                                                           )
                                              )
                                             )
                        )
omega_OO -= numpy.einsum('pi,pj->ij', mo_occ,
                    numpy.einsum('tj,pt->pj', mo_occ, 0.5*fock_RHS_all.alpha_to_numpy(0)))
omega_OO -= ( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_occ)) 
            + numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_occ)).T 
            )

#print(numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda.alpha_to_numpy(0), mo_occ)))
# print("MO occupied energies:")
#print(eocc)
# print()
#eov = -evir + eocc.reshape(-1, 1)
#omega_OO-=eocc*0.5*vlx_dm_oo[i,:]
for i in range(nocc):
    omega_OO[i,:]-=eocc[i]*0.5*vlx_dm_oo[i,:]
    omega_OO[i,i]-=eocc[i]
print(omega_OO)

### $\boldsymbol{\omega}$ entirely in AO basis

In [ ]:
omega_VV_ao = -(numpy.einsum('zr,tz->tr', ovlp,
                         numpy.einsum('nz,tn->tz',
                                   tda_eig_vec_ao, 
                                    0.5*fock_RHS_all.alpha_to_numpy(1).T
                                                                        )
                                                           )
                                              )

# maybe one has to skip teh overlap matrix here already? since one does not transform to MO at all...
omega_VV_ao_wo_s = - (numpy.einsum('nz,tn->tz', tda_eig_vec_ao, 0.5*fock_RHS_all.alpha_to_numpy(1).T))

omega_OV_ao = -(numpy.einsum('rz,mr->mz', ovlp,
                         numpy.einsum('tr,mt->mr',
                               tda_eig_vec_ao, 
                                      0.5*fock_RHS_all.alpha_to_numpy(1).T
                                                                        )
                                                           )
                                              )

omega_OO_ao = -(numpy.einsum('zr,mr->mz', ovlp,
                          numpy.einsum('rp,mp->mr',
                                    tda_eig_vec_ao, 
                                            0.5*fock_RHS_all.alpha_to_numpy(1)
                                                                        )
                                                           )
                                              )
omega_OO_ao -= 0.5*fock_RHS_all.alpha_to_numpy(0)
omega_OO_ao -= ( fock_lambda.alpha_to_numpy(0) + fock_lambda.alpha_to_numpy(0).T )

# Add the "diagonal" parts, transformed to AO
omega_VV_ao -= numpy.matmul(mo_vir, numpy.matmul(ovlp,
                                                 numpy.matmul(numpy.einsum('a,ab->ab', evir, 0.5*vlx_dm_vv),
                                                              numpy.matmul(ovlp, mo_vir).T).T).T)

omega_VV_ao_wo_s -= numpy.matmul(mo_vir, numpy.matmul(numpy.einsum('a,ab->ab', evir, 0.5*vlx_dm_vv), mo_vir.T))

omega_OO_ao -= numpy.matmul(mo_occ, numpy.matmul(numpy.diag(eocc) + numpy.einsum('i,ij->ij', eocc, 0.5*vlx_dm_oo), mo_occ.T))
omega_OV_ao -= (numpy.matmul(mo_occ, numpy.matmul(numpy.einsum('i,ia->ia', eocc, lambda_OV), mo_vir.T))
            + numpy.matmul(mo_vir, numpy.matmul(numpy.einsum('i,ia->ia', eocc, lambda_OV).T, mo_occ.T))
            )



In [ ]:
# Transform those calculated entirely in MO to the AO basis
omega_VV_ao2 = numpy.matmul(mo_vir, numpy.matmul(omega_VV, mo_vir.T))
#omega_VV_wo_e_ao = numpy.matmul(mo_vir, numpy.matmul(omega_VV_wo_e, mo_vir.T))
omega_OV_ao2 = (numpy.matmul(mo_occ, numpy.matmul(omega_OV, mo_vir.T)))
                #+ numpy.matmul(mo_vir, numpy.matmul(omega_OV.T, mo_occ.T)))
omega_OV_ao2 += omega_OV_ao2.T
omega_OO_ao2 = numpy.matmul(mo_occ, numpy.matmul(omega_OO, mo_occ.T))

In [ ]:
print("omega_VV_ao2:")
print(omega_VV_ao2)
print("\nomega_VV_ao_wo_s:")
print(omega_VV_ao_wo_s)
print("\nomega_VV_ao:")
print(omega_VV_ao)
print()
#print("omega_VV_wo_e_ao:")
#print(omega_VV_wo_e_ao)
#print("\nomega_VV_ao_wo_e")
#print(omega_VV_ao_wo_e)

In [ ]:
print("omega_OO_ao2:")
print(omega_OO_ao2)
print("\nomegaOV_ao2:")
print(omega_OV_ao2)